##### Read the notes on the assignment page to check thats whats being requested

## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a field experiment to test the effectiveness of different fundraising letters. They sent out 50,000 fundraising letters to potential donors, randomly assigning each letter to one of three treatments: a standard letter, a matching grant letter, or a challenge grant letter. They published the results of this experiment in the _American Economic Review_ in 2007. The article and supporting data are available from the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action as part of [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).

_to do: expand on the description of the experiment._

This project seeks to replicate their results.


## Data

### Description

_todo: Read the data into R/Python and describe the data_

- First we'll load the data from dta and conduct EDA inorder to better understand the datatypes and distrubtion of data.
- We'll rename the columns we want to make a distribtuion for easier reading.

In [ ]:
import pandas as pd
import pyrsm as rsm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels as sm


In [ ]:
data = pd.read_stata('data/karlan_list_2007.dta')
distr_plot_columns = ['treatment','gave','ask','hpa','ratio','median_hhincome','mrm2','dormant','female']
distr_data_plot = data[distr_plot_columns]
distr_data_plot.rename(columns={
    'hpa': 'Highest Previous Amount',
    'median_hhincome': 'Median Household Income',
    'mrm2': 'Months Since Last Donation',
    'gave': 'Has Previously Donated'
}, inplace=True)



display(data.head(3))


rsm.model.distr_plot(distr_data_plot)


:::: {.callout-note collapse="true"}
### Variable Definitions

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::


### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.

_todo: test a few variables other than the key outcome variables (for example, test months since last donation) to see if the treatment and control groups are statistically significantly different at the 95% confidence level. For at least one variable, perform the test as both t-test (use the formula in the class slides) and separately as a linear regression (regress for example mrm2 on treatment); confirm both methods yield the exact same results. It might be helpful to compare parts of your analysis to Table 1 in the paper. Be sure to comment on your results (hint: why is Table 1 included in the paper)._

::: {.callout-tip appearance="simple" icon=false  title="Variables To Test at the 95% CI"}
`mrm2`, `dormant`,`female`
:::

- We'll conduct a t-test with a 95% confidence interval to ensure and check the months since lost donation variable with a linear regression 
    1) We'll create some list as an efficent coding practice and create a function that calculates the t-values
    3) We'll get the mean and standard deviation of variables and assign them variables for use later


In [ ]:
variables_to_test = ['mrm2', 'dormant', 'female']

def t_test(data, target):
    control_data = data[data['treatment'] == 0]
    treatment_data = data[data['treatment'] == 1]

    control_mean = control_data[target].mean()
    treatment_mean = treatment_data[target].mean()

    control_std = control_data[target].std()
    treatment_std = treatment_data[target].std()

    control_n = len(control_data[target].dropna())
    treatment_n = len(treatment_data[target].dropna())

    se = ((control_std**2 / control_n) + (treatment_std**2 / treatment_n)) ** 0.5
    t_stat = (treatment_mean - control_mean) / se

    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=control_n + treatment_n - 2))

    significance = "significant" if p_value < 0.05 else "not significant"
    

    res = pd.DataFrame([{
        't_stat': round(t_stat, 2),
        'control_mean': round(control_mean, 2),
        'treatment_mean': round(treatment_mean, 2),
        'mean_diff': round(treatment_mean - control_mean, 2),
        'standard_error': round(se, 2),
        'control_n': control_n,
        'treatment_n': treatment_n,
        'p_value':p_value,
        'significance':significance
    }])
    return display(res)
t_test_results = {}

for var in variables_to_test:
    print(f'{var} Results')
    result = t_test(data, var)
    t_test_results[f'variables_{var}'] = result
    

    
reg_data= data[['mrm2','treatment']].dropna()

x = reg_data[['treatment']]
y= reg_data['mrm2']

model= LinearRegression()
model.fit(x,y)
print('\n')
print('mrm2 Linear Regression Results')
print("Intercept:", round(model.intercept_, 2))
print("Treatment coefficient:", round(model.coef_[0], 2))

## Experimental Results

### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 

_todo: make a barplot with two bars. Each bar is the proportion of people who donated. One bar for treatment and one bar for control._

_todo: run a t-test between the treatment and control groups on the binary outcome of whether any charitable donation was made (you can do this as a bivariate linear regression if you want). It may help to confirm your calculations match Table 2a Panel A. Report your statistical results and interpret them in the context of the experiment (e.g., if you found a difference with a small p-value or something that was statistically significant at some threshold, what have you learned about human behavior? Use mostly English words, not numbers or stats, to explain your finding.)_

_todo: run a probit regression where the outcome variable is whether any charitable donation was made and the explanatory variable is assignment to treatment or control._ 

_NOTE: Linear regression results appear replicate Table 3 column 1 in the paper. Probit results do not, despite Table 3 indicating its results come from probit regressions..._

::: {.callout-tip appearance="simple" icon=false  title="Percentage of Donators by Treatment Group"}
:::

In [ ]:
ct = pd.crosstab(data['treatment'],data['gave'],normalize='index')
ct.index = ['Control', 'Treatment']
ct.columns = ['Did Not Give', 'Gave']
display(ct)
ct.plot(kind='bar')
plt.xlabel('Treatment Conditions')
plt.ylabel('Percentage That Donate')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.title('Percentage of Treatment and Control Populations that Donated')

::: {.callout-tip appearance="simple" icon=false  title="T-Test of was a donation made"}
:::

In [ ]:
t_test(data, 'gave')

::: {.callout-tip appearance="simple" icon=false  title="Probit Regression"}
:::

In [ ]:
X = data[['treatment']]
y = data['gave']

# Fit Probit model
probit_model = sm.discrete.discrete_model.Probit(y, X).fit()
print(probit_model.summary())
data['predicted_prob'] = probit_model.predict(X)

# Plot
plt.figure(figsize=(6,4))
means = data.groupby('treatment')['predicted_prob'].mean()

plt.bar(['Control', 'Treatment'], means,)
plt.ylabel("Predicted Probability of Donating")
plt.title("Probit Model: Predicted Probability by Treatment")
plt.ylim(0, means.max() + 0.01)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

_todo: Use a series of t-tests to test whether the size of the match ratio has an effect on whether people donate or not. For example, does the 2:1 match rate lead increase the likelihood that someone donates as compared to the 1:1 match rate? Do your results support the "figures suggest" comment the authors make on page 8?_
::: {.callout-tip appearance="simple" icon=false  title="Match Rate Analysis"}
:::
_todo: Assess the same issue using a regression. Specifically, create the variable `ratio1` then regress `gave` on `ratio1`, `ratio2`, and `ratio3` (or alternatively, regress `gave` on the categorical variable `ratio`). Interpret the coefficients and their statistical precision._
::: {.callout-tip appearance="simple" icon=false  title="Ratio Analysis"}
:::
_todo: Calculate the response rate difference between the 1:1 and 2:1 match ratios and the 2:1 and 3:1 ratios.  Do this directly from the data, and do it by computing the differences in the fitted coefficients of the previous regression. what do you conclude regarding the effectiveness of different sizes of matched donations?_
::: {.callout-tip appearance="simple" icon=false  title="Match Rate Regression Analysis"}
:::

### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.

_todo: Calculate a t-test or run a bivariate linear regression of the donation amount on the treatment status. What do we learn from doing this analysis?_

_todo: now limit the data to just people who made a donation and repeat the previous analysis. This regression allows you to analyze how much respondents donate conditional on donating some positive amount. Interpret the regression coefficients -- what did we learn? Does the treatment coefficient have a causal interpretation?_ 

_todo: Make two plots: one for the treatment group and one for the control. Each plot should be a histogram of the donation amounts only among people who donated. Add a red vertical bar or some other annotation to indicate the sample average for each plot._


## Simulation Experiment

As a reminder of how the t-statistic "works," in this section I use simulation to demonstrate the Law of Large Numbers and the Central Limit Theorem.

Suppose the true distribution of respondents who do not get a charitable donation match is Bernoulli with probability p=0.018 that a donation is made. 

Further suppose that the true distribution of respondents who do get a charitable donation match of any size is Bernoulli with probability p=0.022 that a donation is made.

::: {.callout-tip appearance="simple" icon=false  title="Simulation Using Bernoullui Distrubtion"}
:::

From out bernoulii parameters we can caclulate the mean and the stndard deviation of the distrubtion

In [ ]:
p_no =.018
mu_no_char = p_no
std_no_char = (mu_no_char* (1-mu_no_char))**.5

p = .022
mu_char = p
std_char =  (mu_char * (1-mu_char))**.5

print(f'The mean and standard deviation for the respondents who did not receive a donation match is {mu_no_char,round(std_no_char,2)}')

print(f'The mean and standard deviation for the respondents who did receive a donation match is {mu_char,round(std_char,2)}')

### Law of Large Numbers

_to do:  Simulate 10,000 draws from the control distribution and 10,000 draws from the treatment distribution. You'll then calculate a vector of 10,000 differences, and then you'll plot the cumulative average of that vector of differences. This average will likely be "noisey" when only averaging a few numbers, but should "settle down" and approximate the treatment effect (0.004 = 0.022 - 0.018) as the sample size gets large. Explain the chart to the reader._

In [ ]:
n = 10_000
np.random.seed(38)
no_match_draws = np.random.binomial(1, p_no, size=n)
match_draws = np.random.binomial(1, p, size=n)
differences = match_draws - no_match_draws
cumulative_avg = np.cumsum(differences) / np.arange(1, 10001)
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(cumulative_avg)
plt.axhline(p - p_no, color='red', linestyle='--', label='True Difference')
plt.xlabel('Number of Simulations')
plt.ylabel('Cumulative Avg (Match - No Match)')
plt.title('Law of Large Numbers in Action')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#

### Central Limit Theorem

_to do: Make 4 histograms at sample sizes 50, 200, 500, and 1000.  To do this for a sample size of e.g. 50, take 50 draws from each of the control and treatment distributions, and calculate the average difference between those draws. Then repeat that process 999 more times so that you have 1000 averages. Plot the histogram of those averages. The repeat for the other 3 histograms. Explain this sequence of histograms and its relationship to the central limit theorem to the reader._
::: {.callout-tip appearance="simple" icon=false  title="Central Limit Theorem Demo"}
:::


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

p_no = 0.018
p_match = 0.022

sample_sizes = [50, 200, 500, 1000]
n_sim = 1000

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs = axs.flatten()

for i, n in enumerate(sample_sizes):
    diffs = []

    for _ in range(n_sim):
        no_match = np.random.binomial(1, p_no, size=n)
        match = np.random.binomial(1, p_match, size=n)
        diff = match.mean() - no_match.mean()
        diffs.append(diff)

    axs[i].hist(diffs, bins=30, color='skyblue', edgecolor='black')
    axs[i].axvline(p_match - p_no, color='red', linestyle='--', label='True Difference')
    axs[i].set_title(f"Sample Size = {n}")
    axs[i].set_xlabel("Mean Difference (Match - No Match)")
    axs[i].set_ylabel("Frequency")
    axs[i].legend()

plt.suptitle("Histograms of Sample Mean Differences — Central Limit Theorem in Action")
plt.tight_layout()
plt.show()